**Secventa 1**

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 118.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

**Secventa 2**

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

**Secventa 3** -
  Importarea setului de date

In [ ]:
from datasets import load_dataset
dataset = load_dataset("AnaSeiculescu/fine-tuning-tests")
print(dataset['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14 [00:00<?, ? examples/s]

{'text': '<s>[INST]<<SYS>>Answer only in the Romanian language. Raspunde numai in limba romana.\nRaspunsul trebuie sa fie un plan de dezvoltare a unei aplicatii web sau mobile, in functie de necesitate.\nRaspunsul trebuie sa tina cont de cerintele clientului. Planul de development va fi aplicat de firma de IT. \nDomeniul de activitate al firmei este Software Development.\n<</SYS>>Urmatorul este cerinta clientului:Solicitarea client: Doresc să digitalizeze procesele lor de afaceri și să implementeze un program care să-i ajute să gestioneze rutele, stocul, arhivele furnizorilor, arhivele școlilor și arhivele produselor. 1. Gestiunea rutelor: • Capacitatea de a crea și gestiona rute pentru șoferi 2. Gestiunea stocului: • Capacitatea de a urmări stocul de lapte și corn școlar • Capacitatea de a urmări istoricul vânzărilor 3.Arhive furnizori: • Capacitatea de a stoca informații despre furnizori, cum ar fi numele, adresa, numărul de telefon și datele de contact • Capacitatea de a urmări isto

**Secventa 4** - Definirea parametrilor

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "AnaSeiculescu/fine-tuning-tests"

# Fine-tuned model name
new_model = "Ana-Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 100

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

**Secventa 5** - Antrenarea noului model cu setul de date importat

In [ ]:
# Load dataset (you can process it here)
# [Ana] loaded in a previous step
# dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)
    else:
        print("=" * 80)
        print("Your GPU does not support bfloat16: using float32 instead.")
        print("=" * 80)
        bnb_4bit_compute_dtype = "float32"  # Change compute dtype to float32
        compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

        # Update bnb_config with the new compute dtype
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=use_4bit,
            bnb_4bit_quant_type=bnb_4bit_quant_type,
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=use_nested_quant,
        )

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=False,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

Your GPU supports bfloat16: accelerate training with bf16=True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.295900
50,1.308300
75,0.675700
100,0.244000
125,0.065600
150,0.022400
175,0.014900
200,0.010400
225,0.008500
250,0.007800


TrainOutput(global_step=400, training_loss=0.2935099982842803, metrics={'train_runtime': 3208.4737, 'train_samples_per_second': 0.436, 'train_steps_per_second': 0.125, 'total_flos': 2.9270344728576e+16, 'train_loss': 0.2935099982842803, 'epoch': 100.0})

**Secventa 6**

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

**Secventa 7** - Testarea modelului cu un mesaj din afara setului de fine-tuning

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = """<<SYS>>Answer only in the Romanian language. Raspunde numai in limba romana.
Raspunsul trebuie sa fie un plan de dezvoltare a unei aplicatii web sau mobile, in functie de necesitate.
Raspunsul trebuie sa tina cont de cerintele clientului. Planul de development va fi aplicat de firma de IT.
Domeniul de activitate al firmei este Software Development.<</SYS>>

Urmatorul este cerinta clientului:

 Clientul solicită dezvoltarea unei aplicații de tip Glovo. Specifică următoarele secțiuni:
 ● Sistem de logistică a rider-ului.
 ● Înregistrarea restaurantelor în aplicație.
 ● Primirea comenzilor de către clienți.
 ● Posibilitatea de a adăuga sau elimina produse din meniul restaurantului.
 ● Plata cu cardul.

Pe baza acestora formuleaza un plan de dezvoltare a unei aplicatii web sau
mobile, in functie de necesitate. Descrie aplicatia soclicitata. Ce tehnologii sunt folosite pentru dezvoltarea aplicatiei?
Task-urile concrete si detaliate necesare pentru dezvoltarea aplicatiei, inclusiv cele care nu sunt mentionate de client,
dar sunt necesare, cum ar fi sectiuni financiare, facturare:
Task-uri Adiționale Ne-meneționate de Client, dar Necesare:
 ● Secțiunea financiară: cum restaurantele solicită bani de la administratorii aplicației.
 ● Modalități de plată pentru rideri.
 ● Generarea automată a facturilor și posibilitatea clientului de a descărca factura
 generată.
Planul de dezvoltare are 4 sectiuni principale, care sunt urmatoarele numerotate de la 1 la 4: 1.Scopul documentului,
2.Structura proiectului, 3.Sugestii suplimentare si 4.Pret si timp de implementare.
"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>Answer only in the Romanian language. Raspunde numai in limba romana.
Raspunsul trebuie sa fie un plan de dezvoltare a unei aplicatii web sau mobile, in functie de necesitate.
Raspunsul trebuie sa tina cont de cerintele clientului. Planul de development va fi aplicat de firma de IT. 
Domeniul de activitate al firmei este Software Development.<</SYS>>

Urmatorul este cerinta clientului: 

 Clientul solicită dezvoltarea unei aplicații de tip Glovo. Specifică următoarele secțiuni:
 ● Sistem de logistică a rider-ului.
 ● Înregistrarea restaurantelor în aplicație.
 ● Primirea comenzilor de către clienți.
 ● Posibilitatea de a adăuga sau elimina produse din meniul restaurantului.
 ● Plata cu cardul.

Pe baza acestora formuleaza un plan de dezvoltare a unei aplicatii web sau 
mobile, in functie de necesitate. Descrie aplicatia soclicitata. Ce tehnologii sunt folosite pentru dezvoltarea aplicatiei?
Task-urile concrete si detaliate necesare pentru dezvoltarea aplicatiei, inclusiv 

**Secventa 8**

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

20933

**Secventa 9**

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
# Use offloading to handle memory issues
# import torch

# Clear GPU cache to free up memory
# torch.cuda.empty_cache()

# with torch.device("cuda"):  # Ensure model is loaded on CUDA
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
    # device_map={'':0, 'lm_head': 'cpu'},
    # offload_folder="offload"  # Create a folder to offload weights to disk
)
# model = PeftModel.from_pretrained(base_model, new_model, device_map="auto")
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Now move the merged model to GPU
# model = model.merge_and_unload().to('cuda')

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

**Secventa 10**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

**Secventa 11**

In [ ]:
!huggingface-cli login

model.push_to_hub("AnaSeiculescu/Llama-2-7b-chat-finetune-Veziv-test-03", check_pr=True)

tokenizer.push_to_hub("AnaSeiculescu/Llama-2-7b-chat-finetune-Veziv-test-03",check_pr=True)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AnaSeiculescu/Llama-2-7b-chat-finetune-Veziv-test-03/commit/784fda4909a96ecf1ddf7eb20a861670b4bc0692', commit_message='Upload tokenizer', commit_description='', oid='784fda4909a96ecf1ddf7eb20a861670b4bc0692', pr_url=None, pr_revision=None, pr_num=None)